In [47]:
import torch
import numpy as np
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import time

In [48]:
# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [49]:
# Download and load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = MNIST(root='./data', train=True, download=True, transform=transform)
testset = MNIST(root='./data', train=False, download=True, transform=transform)

# Define data loaders
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

In [50]:
# Initialize CNN and move to GPU
cnn_model = CNN()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cnn_model = cnn_model.to(device)  # Move the initialized model to the GPU if available

# Define criterion and optimizer for CNN
criterion = nn.CrossEntropyLoss()
optimizer_cnn = optim.Adam(cnn_model.parameters(), lr=0.001)

In [51]:
# Functions for evaluation metrics
def calculate_metrics(predictions, true_labels):
    accuracy = (predictions == true_labels).mean()
    # Calculate F1 score (assuming binary classification for simplicity)
    tp = ((predictions == 1) & (true_labels == 1)).sum()
    fp = ((predictions == 1) & (true_labels == 0)).sum()
    fn = ((predictions == 0) & (true_labels == 1)).sum()
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-8)
    return accuracy, f1_score

In [ ]:
# Train CNN
start_time_cnn = time.time()
for epoch in range(5):  # Number of epochs
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer_cnn.zero_grad()
        outputs = cnn_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_cnn.step()
        running_loss += loss.item()
    
    print(f"Epoch [{epoch + 1}/5] - Loss: {running_loss / len(trainloader)}")

end_time_cnn = time.time()

In [ ]:
# Evaluation of CNN
# Calculate predictions
predicted_labels_cnn = []
true_labels_cnn = []
loss_cnn = 0.0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = cnn_model(images)
        _, predicted = torch.max(outputs.data, 1)
        predicted_labels_cnn.extend(predicted.cpu().numpy())
        true_labels_cnn.extend(labels.cpu().numpy())
        loss = criterion(outputs, labels)
        loss_cnn += loss.item()

accuracy_cnn, f1_score_cnn = calculate_metrics(np.array(predicted_labels_cnn), np.array(true_labels_cnn))
loss_cnn /= len(testloader)

print("CNN Metrics:")
print(f"Accuracy: {accuracy_cnn}")
print(f"F1 Score: {f1_score_cnn}")
print(f"Loss: {loss_cnn}")
print(f"Training time: {end_time_cnn - start_time_cnn} seconds")

In [ ]:
# Load pre-trained Faster R-CNN backbone (ResNet50) without the final classification layer
faster_rcnn_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = faster_rcnn_model.roi_heads.box_predictor.cls_score.in_features
faster_rcnn_model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, 10)  # Change the output layer to fit MNIST classes

faster_rcnn_model = faster_rcnn_model.to(device)

# Define criterion and optimizer for Faster R-CNN
params = [p for p in faster_rcnn_model.parameters() if p.requires_grad]
optimizer_faster_rcnn = optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer_faster_rcnn, step_size=3, gamma=0.1)

# Training Faster R-CNN
start_time_faster_rcnn = time.time()
for epoch in range(5):  # Number of epochs
    faster_rcnn_model.train()
    for images, targets in trainloader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = faster_rcnn_model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer_faster_rcnn.zero_grad()
        losses.backward()
        optimizer_faster_rcnn.step()

    lr_scheduler.step()
    print(f"Epoch [{epoch + 1}/5] - Loss: {losses.item()}")

end_time_faster_rcnn = time.time()

# Evaluation of Faster R-CNN
faster_rcnn_model.eval()
predicted_labels_faster_rcnn = []
true_labels_faster_rcnn = []
with torch.no_grad():
    for images, targets in testloader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        outputs = faster_rcnn_model(images)
        for output in outputs:
            predicted_labels_faster_rcnn.extend(output['labels'].cpu().numpy())
        for target in targets:
            true_labels_faster_rcnn.extend(target['labels'].cpu().numpy())

accuracy_faster_rcnn, f1_score_faster_rcnn = calculate_metrics(np.array(predicted_labels_faster_rcnn), np.array(true_labels_faster_rcnn))

print("Faster R-CNN Metrics:")
print(f"Accuracy: {accuracy_faster_rcnn}")
print(f"F1 Score: {f1_score_faster_rcnn}")
print(f"Training time: {end_time_faster_rcnn - start_time_faster_rcnn} seconds")

In [ ]:
# Load pre-trained VGG16
vgg_model = torchvision.models.vgg16(pretrained=True)
vgg_model.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))  # Modify the first layer
vgg_model.classifier[6] = nn.Linear(4096, 10)  # Change the output layer to fit MNIST classes

# Freeze layers except for the final classifier
for param in vgg_model.features.parameters():
    param.requires_grad = False

vgg_model = vgg_model.to(device)

# Define criterion and optimizer for VGG16
criterion = nn.CrossEntropyLoss()
optimizer_vgg = optim.Adam(vgg_model.parameters(), lr=0.001)

# Training VGG16
start_time_vgg = time.time()
for epoch in range(5):  # Number of epochs
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer_vgg.zero_grad()
        outputs = vgg_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_vgg.step()
        running_loss += loss.item()
    
    print(f"Epoch [{epoch + 1}/5] - Loss: {running_loss / len(trainloader)}")

end_time_vgg = time.time()

# Evaluation of VGG16
predicted_labels_vgg = []
true_labels_vgg = []
loss_vgg = 0.0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = vgg_model(images)
        _, predicted = torch.max(outputs.data, 1)
        predicted_labels_vgg.extend(predicted.cpu().numpy())
        true_labels_vgg.extend(labels.cpu().numpy())
        loss = criterion(outputs, labels)
        loss_vgg += loss.item()

accuracy_vgg, f1_score_vgg = calculate_metrics(np.array(predicted_labels_vgg), np.array(true_labels_vgg))
loss_vgg /= len(testloader)

print("VGG16 Metrics:")
print(f"Accuracy: {accuracy_vgg}")
print(f"F1 Score: {f1_score_vgg}")
print(f"Loss: {loss_vgg}")
print(f"Training time: {end_time_vgg - start_time_vgg} seconds")


In [ ]:
# Load pre-trained AlexNet
alexnet_model = torchvision.models.alexnet(pretrained=True)
alexnet_model.features[0] = nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2)  # Modify the first layer
alexnet_model.classifier[6] = nn.Linear(4096, 10)  # Change the output layer to fit MNIST classes

# Freeze layers except for the final classifier
for param in alexnet_model.features.parameters():
    param.requires_grad = False

alexnet_model = alexnet_model.to(device)

# Define criterion and optimizer for AlexNet
criterion = nn.CrossEntropyLoss()
optimizer_alexnet = optim.Adam(alexnet_model.parameters(), lr=0.001)

# Training AlexNet
start_time_alexnet = time.time()
for epoch in range(5):  # Number of epochs
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer_alexnet.zero_grad()
        outputs = alexnet_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_alexnet.step()
        running_loss += loss.item()
    
    print(f"Epoch [{epoch + 1}/5] - Loss: {running_loss / len(trainloader)}")

end_time_alexnet = time.time()

# Evaluation of AlexNet
predicted_labels_alexnet = []
true_labels_alexnet = []
loss_alexnet = 0.0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = alexnet_model(images)
        _, predicted = torch.max(outputs.data, 1)
        predicted_labels_alexnet.extend(predicted.cpu().numpy())
        true_labels_alexnet.extend(labels.cpu().numpy())
        loss = criterion(outputs, labels)
        loss_alexnet += loss.item()

accuracy_alexnet, f1_score_alexnet = calculate_metrics(np.array(predicted_labels_alexnet), np.array(true_labels_alexnet))
loss_alexnet /= len(testloader)

print("AlexNet Metrics:")
print(f"Accuracy: {accuracy_alexnet}")
print(f"F1 Score: {f1_score_alexnet}")
print(f"Loss: {loss_alexnet}")
print(f"Training time: {end_time_alexnet - start_time_alexnet} seconds")